In [4]:
import cv2
import numpy as np

In [5]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [6]:
def detect_bodies(img):
    height, width, channels = img.shape
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 2)

    return img

In [ ]:
mode = input("Enter 'Custom' for Custom input or 'Default' for default input: ")
if mode.lower() == 'custom':
    photo_path = input("Enter the path to the photo: ")
    img = cv2.imread(photo_path)
    output_img = detect_bodies(img)
    cv2.imshow("Pedestrians", output_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
elif mode.lower() == 'default':
    cap = cv2.VideoCapture('test/test.mp4')
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame = detect_bodies(frame)
        cv2.imshow('Pedestrians', processed_frame)
        if cv2.waitKey(1) == 13:
            break
    cap.release()
    cv2.destroyAllWindows()
else:
    print("Error: Invalid mode. Enter 'Custom' or 'Default'.")

NameError: name 'mode' is not defined